In [1]:
import os
import tarfile
import shutil
import dask
import requests
import time
import psutil

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from astrohack import holog
from astrohack.extract_holog import extract_holog
from astrohack.dio import _load_holog_file
from astrohack._utils._io import _make_ant_pnt_dict, _load_pnt_dict
%matplotlib widget
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
from scipy import spatial

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Only a single holography of VLA beam over scans 2,4,6,8,10,12,14,16,18,20,22 (ALMA can map an entire beam in a single scan).
holog_obs_description = {0 :{'scans':[2,4,6,8,10,12,14,16,18,20,22],'ant':{'EA25':['EA08', 'EA04', 'EA12', 'EA27', 'EA19', 'EA07', 'EA21', 'EA03', 'EA11', 'EA05', 'EA01', 'EA09', 'EA24', 'EA22', 'EA14', 'EA18', 'EA02']}}}
holog_obs_description['ddi'] = np.arange(16)

extract_holog(
    ms_name='ea25_cal_small.ms', 
    holog_name='ea25_cal_small', 
    holog_obs_dict=holog_obs_description,
    data_col='DATA',
    subscan_intent='MIXED',
    parallel=False,
    overwrite=True
)

[2023-03-03 11:24:53.639645 Warning]: [extract_holog] Warning, current holography files will be overwritten.
[2023-03-03 11:24:55.081175 System Info]: Opening measurement file ea25_cal_small.ms/DATA_DESCRIPTION
[2023-03-03 11:24:55.145049 System Info]: Processing ddi: 0, scans: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]
Successful readonly open of default-locked table ea25_cal_small.ms/ANTENNA: 8 columns, 28 rows
Successful readonly open of default-locked table ea25_cal_small.ms/OBSERVATION: 9 columns, 1 rows
[2023-03-03 11:25:45.582693 System Info]: [_create_holog_file] Writing holog file to ea25_cal_small.holog.zarr
[2023-03-03 11:25:45.765225 System Info]: Finished extracting holography chunk for ddi: 0 holog_scan_id: 0
[2023-03-03 11:25:45.769876 System Info]: Processing ddi: 1, scans: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]
Successful readonly open of default-locked table ea25_cal_small.ms/ANTENNA: 8 columns, 28 rows
Successful readonly open of default-locked table ea25_cal_small.m

In [3]:
grid_size = None
cell_size = None
grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
chan_tolerance_factor = 0.005
reference_scaling_frequency = None #1.5*10**10 #None #In Hz
#spw_average if True users has to give reference_scaling_frequency
scan_average = True
holog(holog_file='ea25_cal_small.holog.zarr', padding_factor=50, 
      parallel=False,
      grid_size=grid_size, cell_size=cell_size, grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average, chan_tolerance_factor = chan_tolerance_factor,
      reference_scaling_frequency=reference_scaling_frequency,
      scan_average = scan_average,
     )

[2023-03-03 11:28:20.928316 System Info]: Loading holography file ea25_cal_small.holog.zarr ...
[2023-03-03 11:28:20.931392 System Info]: Cell size: [0.00062476 0.00062476] Grid size [31 31]
[2023-03-03 11:28:21.561853 System Info]: [_calculate_aperture_pattern] Calculating aperture illumination pattern ...
[2023-03-03 11:28:21.670079 System Info]: [_holog_chunk] Applying phase correction ...
[2023-03-03 11:28:55.382837 System Info]: [_calculate_aperture_pattern] Calculating aperture illumination pattern ...
[2023-03-03 11:28:55.517450 System Info]: [_holog_chunk] Applying phase correction ...
[2023-03-03 11:29:15.170114 System Info]: [_calculate_aperture_pattern] Calculating aperture illumination pattern ...
[2023-03-03 11:29:15.290877 System Info]: [_holog_chunk] Applying phase correction ...
[2023-03-03 11:29:34.366112 System Info]: [_calculate_aperture_pattern] Calculating aperture illumination pattern ...
[2023-03-03 11:29:34.489536 System Info]: [_holog_chunk] Applying phase corr

In [5]:
%%time
from astrohack.panel import panel
image = 'ea25_cal_small'
#ddi = [1] 
ddi = None

panel(
    holog_image=image, 
    outfile='hack_panel',
    aipsdata=False, 
    unit='mm', 
    save_mask=True,
    save_phase=True,
)

UnboundLocalError: local variable 'telescope' referenced before assignment